# Advent of Code - 2024 - Day 13 - Problem 2

https://adventofcode.com/2024/day/13

## Load Source Data

Load the map data into `DATA`.

In [1]:
f = open("data/day13.txt", "r")
DATA = list(map(str.strip, f.readlines()))
f.close()

DATA = """Button A: X+94, Y+34
Button B: X+22, Y+67
Prize: X=8400, Y=5400

Button A: X+26, Y+66
Button B: X+67, Y+21
Prize: X=12748, Y=12176

Button A: X+17, Y+86
Button B: X+84, Y+37
Prize: X=7870, Y=6450

Button A: X+69, Y+23
Button B: X+27, Y+71
Prize: X=18641, Y=10279"""
DATA = list(map(str.strip, DATA.splitlines()))

## Create Game Class

```
    A * ax + B * bx = px
    A * ay + B * by = py
```

In [5]:
import numpy as np

class Game:

    def __init__(self, button_a_x, button_a_y, button_b_x, button_b_y, prize_x, prize_y):
        self._button_a_x = int(button_a_x)
        self._button_a_y = int(button_a_y)
        self._button_b_x = int(button_b_x)
        self._button_b_y = int(button_b_y)
        self._prize_x = int(prize_x)
        self._prize_y = int(prize_y)

    def __str__(self):
        return (
            f"_button_a_x = {self._button_a_x}, "
            + f"_button_a_y = {self._button_a_y}, "
            + f"_button_b_x = {self._button_b_x}, "
            + f"_button_b_y = {self._button_b_y}, "
            + f"_prize_x = {self._prize_x}, "
            + f"_prize_y = {self._prize_y}"
        )

    def play(self):

        b_presses = min(self._prize_x // self._button_b_x, self._prize_y // self._button_b_y)

        dx = self._prize_x - (b_presses * self._button_b_x)
        dy = self._prize_y - (b_presses * self._button_b_y)

        if dx == 0 and dy == 0:
            return (0, b_presses)

        while True:

            # See if we can win the game by pressing the A button
            #
            a_presses = min(dx // self._button_a_x, dy // self._button_a_y)
            if dx == (a_presses * self._button_a_x) and dy == (a_presses * self._button_a_y):
                return (a_presses, b_presses)

            # "Unpress" the B button once
            #
            dx += self._button_b_x
            dy += self._button_b_y
            b_presses -= 1
            if b_presses < 0:
                return (-1, -1)
            
    def play_quicker(self):

        coefficients = np.array([[self._button_a_x, self._button_b_x], [self._button_a_y, self._button_b_y]])
        results = np.array([self._prize_x, self._prize_y])
        solution = np.linalg.solve(coefficients, results)

        a_presses = solution[0]
        b_presses = solution[1]

        if a_presses != int(a_presses) or b_presses != int(b_presses):
            return (-1, -1)
        
        return (int(a_presses), int(b_presses))

## Parse Data

Sample input:

```
Button A: X+97, Y+12
Button B: X+52, Y+63
Prize: X=10471, Y=4350
```

In [3]:
import re

GAMES = []

BUTTON_REGEX = re.compile("Button [AB]: X\+(\d+), Y\+(\d+)")
PRIZE_REGEX = re.compile("Prize: X=(\d+), Y=(\d+)")

idx = 0
while idx < len(DATA):
    button_a = BUTTON_REGEX.match(DATA[idx + 0])
    button_b = BUTTON_REGEX.match(DATA[idx + 1])
    prize = PRIZE_REGEX.match(DATA[idx + 2])

    button_a_x = button_a.group(1)
    button_a_y = button_a.group(2)
    button_b_x = button_b.group(1)
    button_b_y = button_b.group(2)
    prize_x = prize.group(1)
    prize_y = prize.group(2)

    g = Game(button_a_x, button_a_y, button_b_x, button_b_y, prize_x, prize_y)
    GAMES.append(g)
    idx += 4

len(GAMES)

4

## Play Games

In [4]:
token_count = 0

for game in GAMES:
    print(game)
    solution = game.play()
    print (solution)
    solution = game.play_quicker()
    print (solution)

    coefficients, results = solution
    if coefficients != -1 and results != -1:
        token_count += (coefficients * 3) + results

print(f"token_count = {token_count}")

_button_a_x = 94, _button_a_y = 34, _button_b_x = 22, _button_b_y = 67, _prize_x = 8400, _prize_y = 5400
(80, 40)
[80. 40.]
_button_a_x = 26, _button_a_y = 66, _button_b_x = 67, _button_b_y = 21, _prize_x = 12748, _prize_y = 12176
(-1, -1)
[141.40454076 135.39525284]
_button_a_x = 17, _button_a_y = 86, _button_b_x = 84, _button_b_y = 37, _prize_x = 7870, _prize_y = 6450
(38, 86)
[38. 86.]
_button_a_x = 69, _button_a_y = 23, _button_b_x = 27, _button_b_y = 71, _prize_x = 18641, _prize_y = 10279
(-1, -1)
[244.50163628  65.56989247]
token_count = 480
